In [0]:
!pip install beautifulsoup4==4.7.1

Projet de web-scrapping du menu de Frichti pour savoir quels sont les combinaisons de plats les plus abordables et les plus onéreuses.

In [0]:
#import json
import requests
import pandas as pd
#from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import re
import numpy as np

In [0]:
url = 'https://www.frichti.co/c/plats-cuisines-group/#entrees-1'
r=requests.get(url)
r.content
soup=BeautifulSoup(r.content,"html.parser")
a =soup.get_text()

Filtrage du texte brut et petit nettoyage pour enlever les informations correspondants aux formules et à des morceaux sans pertinence pour mon sujet.

In [0]:
r1 = re.findall(r",\"title\":\"(.*?)\,\"unit\":",a)

In [0]:
r1[:] = [x for x in r1 if not len(x)>375]
r1[:] = [x for x in r1 if ('Cookie') not in x]
r1[:] = [x for x in r1 if ('B2B') not in x]
r1[:] = [x for x in r1 if ('BUNDLE') not in x]

In [0]:
r1=[row.split(',') for row in r1]

In [0]:
for i in range(len(r1)):
  while r1[i][1].startswith('\"type')==False | r1[i][1].startswith('\"cat')==False:
    r1[i][0:2] = [''.join(r1[i][0:2])]

In [0]:
for i in range(len(r1)):
  r1[i][:] = [j for j in r1[i] if ('type') not in j]

Construction d'une base de données des plats proposés avec leur catégorie et leur prix. Une part d'information a été mise de côté lorsque le prix des plats en question n'était pas renseigné dans les infos collectées. Un affinage du scrapping serait nécessaire pour compléter la base de données.

In [0]:
plats=[]
cat=[]
price=[]
for i in range(len(r1)):
  plats.append(r1[i][0])
  cat.append(r1[i][1])
  price.append(r1[i][3])
  

In [9]:
df = pd.DataFrame(plats,columns =['PLATS'])
df['CATEGORY']=cat
df['PRICE']=price
df.CATEGORY = df.CATEGORY.str.strip('\"category\":\"')
df.PRICE = df.PRICE.str.strip('\"price\":\"')
df.PLATS=df.PLATS.str.strip('\"')
df

,PLATS,CATEGORY,PRICE
0,Petit pain tradition,BOULANGERIE,0
1,Dhal d'aubergines,PLAT,0
2,Salade de petits pois à l'oseille,ENTREE,0
3,Le choux chou,ENTREE,350
4,Carottes râpées au gingembre,ENTREE,360
5,Salade gourmande de pois chiches feta & herbes...,ENTREE,370
6,Salade de champignons au ponzu,ENTREE,370
7,Dips ricotta pesto et tomates séchées,ENTREE,390
8,Liberté Égalité Edamame,ENTREE,390
9,Harengs fumés & pommes de terre,ENTREE,420


In [0]:
df1=df[df.PRICE!='0']

In [12]:
df1.CATEGORY=df1.CATEGORY.apply(lambda x: np.where('SNACK' in x,'SNACK',x))
df1.CATEGORY=df1.CATEGORY.apply(lambda x: np.where('JUS' in x,'BOISSON',x))
df1.CATEGORY=df1.CATEGORY.apply(lambda x: np.where('BIERE' in x,'BOISSON',x))
df1.CATEGORY=df1.CATEGORY.apply(lambda x: np.where('BOULANGERIE' in x,'SNACK',x))
df1.CATEGORY=df1.CATEGORY.apply(lambda x: np.where('CREMERIE' in x,'DESSERT',x))
df1.PRICE = pd.to_numeric(df1.PRICE)

df1.PRICE=df1.PRICE.apply(lambda x : x/100)
df1


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,PLATS,CATEGORY,PRICE
3,Le choux chou,ENTREE,3.5
4,Carottes râpées au gingembre,ENTREE,3.6
5,Salade gourmande de pois chiches feta & herbes...,ENTREE,3.7
6,Salade de champignons au ponzu,ENTREE,3.7
7,Dips ricotta pesto et tomates séchées,ENTREE,3.9
8,Liberté Égalité Edamame,ENTREE,3.9
9,Harengs fumés & pommes de terre,ENTREE,4.2
10,Insalata caprese,ENTREE,4.5
11,Dhal d'aubergines,PLAT,6.9
12,Spaghetti cacio e pepe di Inès,PLAT,6.9


une simple étape esthétique pour avoir une vue d'ensemble par catégorie et par prix croissants.

In [32]:
df1.sort_values(['CATEGORY', 'PRICE'], ascending=[True, True], inplace=True)
df1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,PLATS,CATEGORY,PRICE
53,Evian,BOISSON,1.2
54,Coca-Cola,BOISSON,1.5
55,Coca-Cola Zéro,BOISSON,1.5
56,Badoit,BOISSON,1.5
60,Heineken 0.0 25cl,BOISSON,1.6
57,Eau de Coco Bio Cocomi,BOISSON,2.5
58,Jus d'orange Kookabarra (25cl),BOISSON,2.5
59,Pack de 6 Heineken 33cl,BOISSON,6.2
18,Yaourt Grec,DESSERT,2.0
19,Fromage blanc à la fleur d'oranger & au spéculoos,DESSERT,3.2


Etape finale avec construction des compositions les plus et les moins abordables, avec ou sans alcool.

In [22]:
entree=df1[df1.CATEGORY=='ENTREE']
plat=df1[df1.CATEGORY=='PLAT']
dessert=df1[df1.CATEGORY=='DESSERT']
boisson = df1[df1.CATEGORY=='BOISSON']
snack = df1[df1.CATEGORY=='SNACK']
boisson['ALCOOL']=0
boisson.ALCOOL[boisson.PLATS.str.contains('einek')]=1
noalcool=boisson[boisson.ALCOOL==0]


if dessert.PRICE.min() > snack.PRICE.min():
  print('Le repas complet le plus abordable proposé sur Frichti est composé de\n',
        entree.loc[entree['PRICE'] == entree.PRICE.min(), 'PLATS'].iloc[0],',\n',
        plat.loc[plat['PRICE'] == plat.PRICE.min(), 'PLATS'].iloc[0],',\n',
       snack.loc[snack['PRICE'] == snack.PRICE.min(), 'PLATS'].iloc[0],'\n et',
       boisson.loc[boisson['PRICE'] == boisson.PRICE.min(), 'PLATS'].iloc[0],',\n il coûte',
       entree.PRICE.min()+plat.PRICE.min()+snack.PRICE.min()+boisson.PRICE.min(),'€.\n')
else:
  print('Le repas complet le plus abordable proposé sur Frichti est composé de \n',
        entree.loc[entree['PRICE'] == entree.PRICE.min(), 'PLATS'].iloc[0],',\n',
        plat.loc[plat['PRICE'] == plat.PRICE.min(), 'PLATS'].iloc[0],',\n',
       dessert.loc[dessert['PRICE'] == dessert.PRICE.min(), 'PLATS'].iloc[0],'\n et',
       boisson.loc[boisson['PRICE'] == boisson.PRICE.min(), 'PLATS'].iloc[0],',\n il coûte',
       entree.PRICE.min()+plat.PRICE.min()+dessert.PRICE.min()+boisson.PRICE.min(),'€.\n')


if dessert.PRICE.max() > snack.PRICE.min():
  if boisson.PRICE.max() >= noalcool.PRICE.max():
    print('Le repas complet le plus onéreux proposé sur Frichti est composé de\n',
        entree.loc[entree['PRICE'] == entree.PRICE.max(), 'PLATS'].iloc[0],',\n',
        plat.loc[plat['PRICE'] == plat.PRICE.max(), 'PLATS'].iloc[0],',\n',
       snack.loc[snack['PRICE'] == snack.PRICE.max(), 'PLATS'].iloc[0],',\n',
       boisson.loc[boisson['PRICE'] == boisson.PRICE.max(), 'PLATS'].iloc[0],',\n il coûte',
       entree.PRICE.max()+plat.PRICE.max()+snack.PRICE.max()+boisson.PRICE.max(),
          '€.\n \n Pour un repas sans alcool, l\'offre la plus consistante est composée de\n',
       entree.loc[entree['PRICE'] == entree.PRICE.max(), 'PLATS'].iloc[0],',\n',
        plat.loc[plat['PRICE'] == plat.PRICE.max(), 'PLATS'].iloc[0],',\n',
       snack.loc[snack['PRICE'] == snack.PRICE.max(), 'PLATS'].iloc[0],',\n',
       noalcool.loc[noalcool['PRICE'] == noalcool.PRICE.max(), 'PLATS'].iloc[0],',\n il coûte',
       entree.PRICE.max()+plat.PRICE.max()+snack.PRICE.max()+noalcool.PRICE.max(),'€.\n')
  else:
    print('Pour un repas sans alcool, l\'offre la plus consistante est composée de\n',
       entree.loc[entree['PRICE'] == entree.PRICE.max(), 'PLATS'].iloc[0],',\n',
        plat.loc[plat['PRICE'] == plat.PRICE.max(), 'PLATS'].iloc[0],',\n',
       snack.loc[snack['PRICE'] == snack.PRICE.max(), 'PLATS'].iloc[0],',\n et',
       noalcool.loc[noalcool['PRICE'] == noalcool.PRICE.max(), 'PLATS'].iloc[0],',\n il coûte',
       entree.PRICE.max()+plat.PRICE.max()+snack.PRICE.max()+noalcool.PRICE.max(),'€.\n')

Le repas complet le plus abordable proposé sur Frichti est composé de
 Le choux chou ,
 Dhal d'aubergines ,
 Brioche moelleuse au chocolat 
 et Evian ,
 il coûte 13.4 €.

Le repas complet le plus onéreux proposé sur Frichti est composé de
 Insalata caprese ,
 Couscous d'été ,
 15 petits Goulibeur tradition ,
 Pack de 6 Heineken 33cl ,
 il coûte 21.9 €.
 
 Pour un repas sans alcool, l'offre la plus consistante est composée de
 Insalata caprese ,
 Couscous d'été ,
 15 petits Goulibeur tradition ,
 Eau de Coco Bio Cocomi ,
 il coûte 18.2 €.



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:8682: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/usr/local/lib/python3.6/dist-packages/IPython/core/inte

Reprise de tout le code précédent pour le transformer en programme qui sélectionne entre 2 choix : repas abordable ou onéreux.

In [0]:
def frichti():

  url = 'https://www.frichti.co/c/plats-cuisines-group/#entrees-1'
  r=requests.get(url)
  r.content
  soup=BeautifulSoup(r.content,"html.parser")
  a =soup.get_text()
  r1 = re.findall(r",\"title\":\"(.*?)\,\"unit\":",a)
  r1[:] = [x for x in r1 if not len(x)>375]
  r1[:] = [x for x in r1 if ('Cookie') not in x]
  r1[:] = [x for x in r1 if ('B2B') not in x]
  r1[:] = [x for x in r1 if ('BUNDLE') not in x]
  r1=[row.split(',') for row in r1]
  for i in range(len(r1)):
    while r1[i][1].startswith('\"type')==False | r1[i][1].startswith('\"cat')==False:
      r1[i][0:2] = [''.join(r1[i][0:2])]
  for i in range(len(r1)):
    r1[i][:] = [j for j in r1[i] if ('type') not in j]
  plats=[]
  cat=[]
  price=[]
  for i in range(len(r1)):
    plats.append(r1[i][0])
    cat.append(r1[i][1])
    price.append(r1[i][3])

  df = pd.DataFrame(plats,columns =['PLATS'])
  df['CATEGORY']=cat
  df['PRICE']=price
  df.CATEGORY = df.CATEGORY.str.strip('\"category\":\"')
  df.PRICE = df.PRICE.str.strip('\"price\":\"')
  df.PLATS=df.PLATS.str.strip('\"')
  df1=df[df.PRICE!='0']
  df1.CATEGORY=df1.CATEGORY.apply(lambda x: np.where('SNACK' in x,'SNACK',x))
  df1.CATEGORY=df1.CATEGORY.apply(lambda x: np.where('JUS' in x,'BOISSON',x))
  df1.CATEGORY=df1.CATEGORY.apply(lambda x: np.where('BIERE' in x,'BOISSON',x))
  df1.CATEGORY=df1.CATEGORY.apply(lambda x: np.where('BOULANGERIE' in x,'SNACK',x))
  df1.CATEGORY=df1.CATEGORY.apply(lambda x: np.where('CREMERIE' in x,'DESSERT',x))
  df1.PRICE = pd.to_numeric(df1.PRICE)
  df1.PRICE=df1.PRICE.apply(lambda x : x/100)
  entree=df1[df1.CATEGORY=='ENTREE']
  plat=df1[df1.CATEGORY=='PLAT']
  dessert=df1[df1.CATEGORY=='DESSERT']
  boisson = df1[df1.CATEGORY=='BOISSON']
  snack = df1[df1.CATEGORY=='SNACK']
  boisson['ALCOOL']=0
  boisson.ALCOOL[boisson.PLATS.str.contains('einek')]=1
  noalcool=boisson[boisson.ALCOOL==0]


  message = 'Souhaitez-vous une suggestion pour /n une offre abordable (tapez 1) \n une offre plus onéreuse (tapez 2) :'
  error_mess = 'Seulement les chiffres 1 ou 2 sont des réponses valides Merci de recommencer.'
  choice = input(message)
  while choice.isdigit() == False:
    print(error_mess)
    choice = input(message)
  choice = int(choice)
  while (choice !=1) & (choice != 2) :
    print(error_mess)
    choice = int(input(message))

  if choice == 1:
    if dessert.PRICE.min() > snack.PRICE.min():
      print('Le repas complet le plus abordable proposé sur Frichti est composé de\n',
            entree.loc[entree['PRICE'] == entree.PRICE.min(), 'PLATS'].iloc[0],',\n',
            plat.loc[plat['PRICE'] == plat.PRICE.min(), 'PLATS'].iloc[0],',\n',
           snack.loc[snack['PRICE'] == snack.PRICE.min(), 'PLATS'].iloc[0],'\n et',
           boisson.loc[boisson['PRICE'] == boisson.PRICE.min(), 'PLATS'].iloc[0],',\n il coûte',
           entree.PRICE.min()+plat.PRICE.min()+snack.PRICE.min()+boisson.PRICE.min(),'€.\n')
    else:
      print('Le repas complet le plus abordable proposé sur Frichti est composé de \n',
            entree.loc[entree['PRICE'] == entree.PRICE.min(), 'PLATS'].iloc[0],',\n',
            plat.loc[plat['PRICE'] == plat.PRICE.min(), 'PLATS'].iloc[0],',\n',
           dessert.loc[dessert['PRICE'] == dessert.PRICE.min(), 'PLATS'].iloc[0],'\n et',
           boisson.loc[boisson['PRICE'] == boisson.PRICE.min(), 'PLATS'].iloc[0],',\n il coûte',
           entree.PRICE.min()+plat.PRICE.min()+dessert.PRICE.min()+boisson.PRICE.min(),'€.\n')

  if choice == 2:
    if dessert.PRICE.max() > snack.PRICE.min():
      message2 = 'Souhaitez-vous une suggestion pour /n une offre avec alcool (tapez 1) \n ou sans (tapez 2) :'
      error_mess2 = 'Seulement les chiffres 1 ou 2 sont des réponses valides Merci de recommencer.'
      choice2 = input(message2)
      while choice2.isdigit() == False:
        print(error_mess2)
        choice2 = input(message2)
      choice2 = int(choice2)
      while (choice2 !=1) & (choice2 != 2) :
        print(error_mess2)
        choice2 = int(input(message2))
      if choice2==1:
        print('Le repas complet le plus onéreux proposé sur Frichti est composé de\n',
            entree.loc[entree['PRICE'] == entree.PRICE.max(), 'PLATS'].iloc[0],',\n',
            plat.loc[plat['PRICE'] == plat.PRICE.max(), 'PLATS'].iloc[0],',\n',
           snack.loc[snack['PRICE'] == snack.PRICE.max(), 'PLATS'].iloc[0],',\n',
           boisson.loc[boisson['PRICE'] == boisson.PRICE.max(), 'PLATS'].iloc[0],',\n il coûte',
           entree.PRICE.max()+plat.PRICE.max()+snack.PRICE.max()+boisson.PRICE.max(),
              '€.\n')
      else:
        print('Pour un repas sans alcool, l\'offre la plus consistante est composée de\n',
           entree.loc[entree['PRICE'] == entree.PRICE.max(), 'PLATS'].iloc[0],',\n',
            plat.loc[plat['PRICE'] == plat.PRICE.max(), 'PLATS'].iloc[0],',\n',
           snack.loc[snack['PRICE'] == snack.PRICE.max(), 'PLATS'].iloc[0],',\n et',
           noalcool.loc[noalcool['PRICE'] == noalcool.PRICE.max(), 'PLATS'].iloc[0],',\n il coûte',
           entree.PRICE.max()+plat.PRICE.max()+snack.PRICE.max()+noalcool.PRICE.max(),'€.\n')


essai réussi avec le programme de sélection, vous pouvez le tester à votre tour : 

In [31]:
frichti()

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/

Souhaitez-vous une suggestion pour /n une offre abordable (tapez 1) 
 une offre plus onéreuse (tapez 2) :2
Souhaitez-vous une suggestion pour /n une offre avec alcool (tapez 1) 
 ou sans (tapez 2) :z
Seulement les chiffres 1 ou 2 sont des réponses valides Merci de recommencer.
Souhaitez-vous une suggestion pour /n une offre avec alcool (tapez 1) 
 ou sans (tapez 2) :1
Le repas complet le plus onéreux proposé sur Frichti est composé de
 Legumi Grigliati e Bufala ,
 Couscous d'été ,
 15 petits Goulibeur tradition ,
 Pack de 6 Heineken 33cl ,
 il coûte 22.3 €.

